# Import Library

In [45]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "hactiv8-yosia" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


In [46]:
from scipy import stats
import pandas as pd


# Problem Statement

Problem Statement : Meningkatkan jumlah user retention sebanyak 5% dalam waktu 6 bulan.

# 1

1.   Berapa persen jumlah user yang retention dari Januari - Juni 2022 dari total keseluruhan user yang ada? (Asumsi definisi retention minimal melakukan transaksi 6 kali selama periode) Apa kesimpulan/insight yang bisa kamu berikan?

In [47]:
df1 = client.query('''
with T as
(
  SELECT user_id, COUNT(created_at)
  FROM `bigquery-public-data.thelook_ecommerce.orders`
  WHERE created_at >= '2022-01-01' and created_at <= '2022-06-30'
  GROUP BY user_id
  HAVING COUNT(created_at) >= 3
), R as
(
  select distinct id
  from `bigquery-public-data.thelook_ecommerce.users`
), RU as
(
  select (select count(*) from T) as retention_user, (select count(*) from R) as all_user
)
select retention_user, all_user, (retention_user/all_user*100) as percentage
from RU
''').to_dataframe()
df1

,retention_user,all_user,percentage
0,1047,100000,1.047


Dari hasil di atas kita bisa melihat bahwa dari jumlah seluruh users yang mencapai 100000, hanya terdapat 1047 user yang retention pada periode Januari - Juni 2022 atau hanya sekitar 1% dari seluruh user yang artinya user retention dari TheLook sangat sedikit

# 2

2.   Berikan informasi rata-rata, minimum, dan maksimum jumlah transaksi yang dilakukan oleh user loyal? (Tampilkan hanya dalam satu dataframe dan berdasarkan satu query select SQL) Informasi apa yang bisa kamu berikan?

In [48]:
df2 = client.query('''
with T as
(
  SELECT user_id, COUNT(created_at) as jumlah_transaksi
  FROM `bigquery-public-data.thelook_ecommerce.orders`
  WHERE created_at >= '2022-01-01' and created_at <= '2022-06-30'
  GROUP BY user_id
  HAVING COUNT(created_at) >= 3
)
select avg(jumlah_transaksi) as average, min(jumlah_transaksi) as min, max(jumlah_transaksi) as max
from T
''').to_dataframe()
df2

,average,min,max
0,3.176695,3,4


Dari hasil di atas kita bisa melihat bahwa user loyal dalam periode Januari - Juni 2022 rata-rata melakukan transaksi sebanyak 3 kali dengan jumlah transaksi terbanyak adalah 4 kali transaksi dalam rentang waktu Januari - Juni 2022

# 3

3. Berikan informasi rata-rata, minimum, dan maksimum uang yang dihabiskan oleh user loyal? (Tampilkan hanya dalam satu dataframe dan berdasarkan satu query select SQL) Informasi apa yang bisa kamu berikan?

In [49]:
df3 = client.query('''
with T as
(
  SELECT user_id, COUNT(created_at) as jumlah_transaksi
  FROM `bigquery-public-data.thelook_ecommerce.orders`
  WHERE created_at >= '2022-01-01' and created_at <= '2022-06-30'
  GROUP BY user_id
  HAVING COUNT(created_at) >= 3
), K as
(
  SELECT user_id, order_id
  FROM `bigquery-public-data.thelook_ecommerce.orders`
  WHERE created_at >= '2022-01-01' and created_at <= '2022-06-30'
), G as
(
  select T.user_id, K.order_id
  FROM K
  INNER JOIN T
  ON K.user_id = T.user_id
), U as
(
  select order_id, product_id
  from `bigquery-public-data.thelook_ecommerce.order_items`
), O as
(
  select G.order_id, U.product_id
  from U
  inner join G
  on U.order_id = G.order_id
), P as
(
  select id, retail_price
  from  `bigquery-public-data.thelook_ecommerce.products`
), L as
(
  select O.product_id, P.retail_price
  FROM P
  INNER JOIN O
  ON P.id = O.product_id
)
select avg(retail_price) as average, min(retail_price) as min, max(retail_price) as max
from L
''').to_dataframe()
df3

,average,min,max
0,59.46082,1.72,903.0


Dari hasil di atas kita bisa melihat bahwa rata-rata user loyal menghabiskan uang sebanyak 59.46 dollar dimana transaksi terkecil adalah 1.72 dollar dan transaksi tertinggi adalah 903 dollar. Karena rata-ratanya lebih dekat ke min, artinya lebih banyak user yang membeli barang murah daripada user yang membeli barang mahal.

# 4

4. Berasal dari sumber trafik mana saja kebanyakan user loyal akses platform The Look? Informasi apa yang bisa kamu berikan?

In [50]:
df4 = client.query('''
with T as
(
  SELECT user_id, COUNT(created_at) as jumlah_transaksi
  FROM `bigquery-public-data.thelook_ecommerce.orders`
  WHERE created_at >= '2022-01-01' and created_at <= '2022-06-30'
  GROUP BY user_id
  HAVING COUNT(created_at) >= 3
), J as
(
  SELECT id, traffic_source
  from `bigquery-public-data.thelook_ecommerce.users`
)
select J.traffic_source, count(J.traffic_source) as jumlah
FROM J
INNER JOIN T
ON J.id = T.user_id
GROUP BY J.traffic_source
ORDER BY jumlah DESC
''').to_dataframe()
df4

,traffic_source,jumlah
0,Search,724
1,Organic,154
2,Facebook,66
3,Email,63
4,Display,40


Dari hasil di atas kita bisa melihat bahwa kebanyakan user loyal dari The Look mengakses platform The look dari search atau pencarian yang artinya The Look sudah cukup populer sehingga banyak user yang mencari tahu tentang The Look sampai akhirnya menjadi user loyal.

# 5

5. Apakah ada hubungan antara sumber trafik dengan loyal atau tidaknya seorang user? (Gunakan chi-square test untuk menguji hubungan itu) Hint: Output tabel query hanya ada dua kolom yaitu kolom 'trafic source' dan 'loyal' (kolom 'loyal' berisikan ya dan tidak saja

In [51]:
df5 = client.query('''
with T as
(
  SELECT user_id, COUNT(created_at) as jumlah_transaksi
  FROM `bigquery-public-data.thelook_ecommerce.orders`
  WHERE created_at >= '2022-01-01' and created_at <= '2022-06-30'
  GROUP BY user_id
  HAVING COUNT(created_at) >= 3
), J as
(
  SELECT id, traffic_source
  from `bigquery-public-data.thelook_ecommerce.users`
), P as
(
SELECT J.traffic_source, count(J.traffic_source) as jumlah,
CASE WHEN count(J.traffic_source) >= 100 THEN 'YES'
ELSE 'NO'
END AS loyal
FROM J
INNER JOIN T
ON J.id = T.user_id
GROUP BY J.traffic_source
)
select traffic_source, loyal
from P
''').to_dataframe()
df5

,traffic_source,loyal
0,Organic,YES
1,Search,YES
2,Email,NO
3,Facebook,NO
4,Display,NO


In [52]:
contingency_table=pd.crosstab(df5["traffic_source"],df5["loyal"])
contingency_table

loyal,NO,YES
traffic_source,,
Display,1,0
Email,1,0
Facebook,1,0
Organic,0,1
Search,0,1


In [53]:
stat, p, dof, expected = stats.chi2_contingency(contingency_table)
print('p=%.3f' % (p))
if p > 0.05:
    print('Probably independent')
else:
    print('Probably dependent')

p=0.287
Probably independent


Dari hasil di atas kita bisa melihat ternyata tidak ada hubungan antara traffic source dan juga loyal/tidaknya suatu user. Artinya user loyal bisa berasal dari traffic mana saja.

# Poin Analisis

Berikan kesimpulan dari laporan/informasi yang dibuat berdasarkan problem statement/poin penjabaran menggunakan bahasa awam! (boleh dalam beberapa paragraf)



Dari hasil di atas kita bisa mengambil beberapa informasi penting diantara lain adalah ternyata user loyal dari The Look pada periode Januari - Juni 2022 sangat sedikit dan rata-rata hanya melakukan 3 kali transaksi pada periode itu. Selain itu kita juga bisa melihat bahwa rata-rata user loyal menghabiskan uang sebanyak 59 dollar. User loyal ternyata bisa mengakses platform The Look kebanyakan dari search atau pencarian yang artinya banyak user yang penasaran dan mencari tau tentang The Look. Kesimpulannya The Look merupakan salah satu ecommerce yang banyak dicari oleh orang namun hanya memiliki sedikit user loyal yang menurut saya bisa ditingkatkan dengan cara mengurangi harga atau memberi diskon karena banyak user yang retention membeli barang yang tidak terlalu mahal, dan juga dengan menambah fitur menarik atau meningkatkan marketing maka saya yakin akan lebih banyak user yang mencari tentang The Look dan kemungkinan bisa jadi user loyal.

# Poin Pertanyaan

1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

Seharusnya problem statement di awal bisa tercapai karena jika kita bisa mengurangi harga barang dan meningkatkan iklan maka dengan yakin user retention bisa meningkat sebanyak 5% dari saat ini.

2. Apa yang dimaksud dengan customer/user retention?

Customer/User retention adalah sebutan untuk tingkat customer yang bertahan atau tetap menggunakan suatu platform atau dalam kasus ecommerce adalah melakukan transaksi lebih dari sekali.

3. Jika platform The Look membuat fitur tambahan, apakah kamu akan merekomendasikan/memprioritaskan kepada user loyal? apa pertimbanganmu?

Menurut saya karena user loyal jumlahnya sedikit jika dibandingkan dengan user biasa, jadi seharusnya jika akan membuat fitur tambahan kita bisa lebih fokus ke user biasa sehingga user biasa tersebut bisa lebih tertarik dengan The Look karena fitur barunya dan menjadi user loyal.